In [1]:
!pip install backtrader
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/cfrm523/Final_project')
from datetime import datetime

<ipython-input-2-b11105c7cb64>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Mounted at /content/drive


In [126]:
data = pd.read_csv('crypto_mar_april.csv')
data['date'] = pd.to_datetime(data['date'])
data = data.drop('Unnamed: 0', axis=1)

df = data.loc[data['date'] >= datetime(2024,3,1,0,0,0)]
df = df.loc[df['date'] <= datetime(2024,3,30,0,0,0)]
df = df.drop_duplicates()
df = df.reset_index(drop=True)

btc = df.loc[df['ticker']=='BTC-USD']
btc = btc.reset_index(drop=True)
btc = btc.set_index('date')

eth = df.loc[df['ticker']=='ETH-USD']
eth = eth.reset_index(drop=True)
eth = eth.set_index('date')

In [127]:
from arch.unitroot import ADF
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from sklearn.linear_model import LinearRegression

# Create a Stratey
class TestStrategy(bt.Strategy):

    params = (
        ('history', 120),
        ('lookback', 2)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy '''
        if dt is None:
            # Access the datetime index from the current line in the data series
            dt = self.datas[0].datetime.datetime(0)

        # Check if dt is still a float (the internal representation for Backtrader), and convert it if needed
        if isinstance(dt, float):
            # Convert backtrader float date to datetime
            dt = bt.num2date(dt)

        # Format datetime object to string
        dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')
        print('%s, %s' % (dt_str, txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.data0 = self.datas[0].close
        self.data1 = self.datas[1].close
        self.cointegrated = False
        self.upper_bound = None
        self.lower_bound = None
        self.model_built = False
        self.coefficients = None

    #trading during best time
    def next(self):
        current_datetime = self.datas[0].datetime.datetime(0)
        current_hour = current_datetime.hour
        current_minute = current_datetime.minute
        if 6 <= current_hour < 8: #this is inherently rolling
          if len(self.data0) >= self.params.history and len(self.data1) >= self.params.history: #check if there is sufficient data for looking back
            series1 = pd.Series(self.data0.get(size=self.params.history))
            series2 = pd.Series(self.data1.get(size=self.params.history))
            series1_diff = series1.diff().dropna()  # taking the diff of both series
            series2_diff = series2.diff().dropna()
            tickers = [series1_diff, series2_diff]
            #######################################
            if self.model_built == False: #we only build the model once for the time period
              for i in range(len(tickers)): #iterate through series and check for stationarity
                adf = ADF(tickers[i])
                pp = PhillipsPerron(tickers[i])
                kpss = KPSS(tickers[i])

                if adf.pvalue < 0.05 and pp.pvalue < 0.05 and kpss.pvalue > 0.10: #check weather pass all the tests
                  self.cointegrated = True
                else:
                  self.cointegrated = False
                self.log(f'Checking series stationarity: ADF P-Value: {adf.pvalue}, PP P-Value: {pp.pvalue}, KPSS P-Value: {kpss.pvalue}')

              #we want to define the model once and trade on that model for the time period
              if self.cointegrated == True: #2nd step, fit a linear model and create a spread
                lin_model = LinearRegression()
                lin_model.fit(series1.values.reshape(-1,1), series2.values)  # Fit model to the differenced data
                spread = 1*series1 - (lin_model.coef_[0] * series2) #value of the spread for model
                mean = np.mean(spread)#mean
                self.upper_bound = mean + 1.5*np.std(spread)
                self.lower_bound = mean - 1.5*np.std(spread)
                self.coefficients = [1] + list(lin_model.coef_)
                self.model_built = True
                self.log(f'Upper bound {self.upper_bound}, Lower bound {self.lower_bound}, coefs:{self.coefficients}')
            ######################################
          #we check if upper or lower bound is crossed to enter a trade
            elif self.model_built == True:
              #need to calculate the value of the spread
              spread_val = self.data0[0] - self.data1[0]*self.coefficients[1]
              #identify if not in position
              if not self.position:
                #get into a long
                if spread_val < self.lower_bound:
                  self.log('Buy Created, %.2f' % spread_val)
                  self.buy(self.datas[0], size=1)
                  self.buy(self.datas[1], size=self.coefficients[1])
                #get into a short
                elif spread_val > self.upper_bound:
                  self.log('Sell Created, %.2f' % spread_val)
                  self.sell(self.datas[0], size=1)
                  self.sell(self.datas[1], size=self.coefficients[1])
              #identify if in position
              elif self.position:
                position_size = self.position.size
                #check if currently in long and hit our exit (upper boundary)
                if position_size > 0 and spread_val > self.upper_bound:
                    self.log('Close existing Long position, %.2f' % spread_val)
                    self.close()
                #check if currently in short and hit our exit (lower boundary)
                elif position_size < 0 and spread_val < self.lower_bound:
                    self.log('Close existing Short position, %.2f' % spread_val)
                    self.close()

        #reset our model after 6 and close out any positions
        if current_hour == 8 and current_minute == 0: #always close out trades
          self.model_built = False
          if self.position:
            self.log("Closing all positions: ")
            self.close()

In [128]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
btc_bt = bt.feeds.PandasData(dataname=btc)
eth_bt = bt.feeds.PandasData(dataname=eth)
cerebro.adddata(btc_bt)
cerebro.adddata(eth_bt)
initial_cash = 100000
cerebro.broker.set_cash(initial_cash)
cerebro.broker.setcommission(commission=0.0)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio", riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")

In [129]:
# Run backtest
results = cerebro.run()
strategy = results[0]

# Extract metrics from analyzers
trade_analyzer = strategy.analyzers.trade_analyzer.get_analysis()
sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis()
drawdown = strategy.analyzers.drawdown.get_analysis()
returns = strategy.analyzers.returns.get_analysis()

2024-03-01 06:00:00, Checking series stationarity: ADF P-Value: 3.556103617574946e-15, PP P-Value: 3.180492129860898e-17, KPSS P-Value: 0.03246938859027307
2024-03-01 06:00:00, Checking series stationarity: ADF P-Value: 1.5900957791171805e-06, PP P-Value: 7.367981235060634e-18, KPSS P-Value: 0.24342600819805318
2024-03-01 06:00:00, Upper bound 61470.81314678685, Lower bound 61122.71799277306, coefs:[1, 0.0259038467121626]
2024-03-01 06:16:00, Sell Created, 61573.78
2024-03-01 08:00:00, Closing all positions: 
2024-03-02 06:00:00, Checking series stationarity: ADF P-Value: 1.0590080063907913e-19, PP P-Value: 1.0381958940559993e-20, KPSS P-Value: 0.6538670017232036
2024-03-02 06:00:00, Checking series stationarity: ADF P-Value: 1.2076545943300465e-25, PP P-Value: 7.85683352723287e-27, KPSS P-Value: 0.3756563499131484
2024-03-02 06:00:00, Upper bound 62034.03944009025, Lower bound 61668.08877235029, coefs:[1, 0.08073012937735784]
2024-03-02 06:21:00, Sell Created, 62044.73
2024-03-02 07:0

In [130]:
# Calculate custom metrics
total_net_profit = cerebro.broker.getvalue() - initial_cash
total_gross_profit = trade_analyzer.won.pnl.total
total_gross_loss = trade_analyzer.lost.pnl.total
total_number_trades = trade_analyzer.total.total
percent_profitable = (trade_analyzer.won.total / trade_analyzer.total.total) * 100
winning_trades = trade_analyzer.won.total
loosing_trades = trade_analyzer.lost.total
avg_trade_net_profit = total_net_profit / trade_analyzer.total.total
avg_winning_trade = trade_analyzer.won.pnl.average
avg_losing_trade = trade_analyzer.lost.pnl.average
ratio_avg_win_loss = avg_winning_trade/avg_losing_trade
largest_winning_trade = trade_analyzer.won.pnl.max
largest_losing_trade = trade_analyzer.lost.pnl.max
max_consecutive_winning_trades = trade_analyzer.streak.won.longest
max_consecutive_losing_trades = trade_analyzer.streak.lost.longest
avg_bars_in_total_trades = trade_analyzer.len.total
avg_bars_in_winning_trades = trade_analyzer.len.won.total
avg_bars_in_losing_trades = trade_analyzer.len.lost.total
max_drawdown = drawdown.max.moneydown

In [131]:
final_value = cerebro.broker.getvalue()
profit = final_value - initial_cash

print(f"Initial Cash: ${initial_cash}")
print(f"Final Portfolio Value: ${final_value}")
print(f"Profit / Loss: ${profit}")

Initial Cash: $100000
Final Portfolio Value: $95407.25085364861
Profit / Loss: $-4592.749146351387


In [132]:
print(profit, total_gross_profit, total_gross_loss, total_gross_loss + total_gross_profit)

-4592.749146351387 3810.4589766338154 -8472.350000000013 -4661.891023366197


In [148]:
metrics = {
        'spread': ['BTC-ETH'],
        'Total Net Profit': [total_net_profit],
        'Gross Profit': [total_gross_profit],
        'Gross Loss': [total_gross_loss],
        'Percent Profitable': [percent_profitable],
        'Winning Trades': [trade_analyzer.won.total],
        'Losing Trades': [trade_analyzer.lost.total],
        'Avg. Trade Net Profit': [avg_trade_net_profit],
        'Avg. Winning Trade': [avg_winning_trade],
        'Avg. Losing Trade': [avg_losing_trade],
        'Ratio Avg. Win:Avg. Loss': [ratio_avg_win_loss],
        'Largest Winning Trade': [largest_winning_trade],
        'Largest Losing Trade': [largest_losing_trade],
        'Max. Consecutive Winning Trades': [max_consecutive_winning_trades],
        'Max. Consecutive Losing Trades': [max_consecutive_losing_trades],
        'Avg. Bars in Total Trades': [avg_bars_in_total_trades],
        'Avg. Bars in Winning Trades': [avg_bars_in_winning_trades],
        'Avg. Bars in Losing Trades': [avg_bars_in_losing_trades],
        'Max. Drawdown': [max_drawdown]
    }

In [150]:
pd.DataFrame.from_dict(metrics).T

,0
spread,BTC-ETH
Total Net Profit,-4592.749146
Gross Profit,3810.458977
Gross Loss,-8472.35
Percent Profitable,46.153846
Winning Trades,18
Losing Trades,20
Avg. Trade Net Profit,-117.762799
Avg. Winning Trade,211.692165
Avg. Losing Trade,-423.6175
